# Bulk Assign a Glossary Term to Assets

What does the notebook do?

1. Reads the glossary
1. Finds the term guid based on the GLOSSARY_TERM_FORMAL_NAME
1. Converts the CSV into an array of GUIDs
1. Assigns the term to all the assets from the CSV

In [ ]:
# ENVIRONMENT VARIABLES
# Note #1: You can find a terms formal name by navigating to the term within the Microsoft Purview Governance Portal
# Note #2: The script expects the first column in the CSV to be a GUID
%env PURVIEW_NAME=pvdemoxv2hu-pv
GLOSSARY_TERM_FORMAL_NAME = "Workplace Analytics_Conflicting meeting"
PATH_TO_CSV = "C:/Users/tarifat/Downloads/GitHub/purviewcli/samples/csv/assets.csv"

In [ ]:
# Helper Methods
import json
def getJSON(raw_output):
    output = ''.join(raw_output)
    json_obj = json.loads(output)
    return json_obj

In [ ]:
# 1. Get Glossary
glossary = !pv glossary read
glossary = getJSON(glossary)

In [ ]:
# 2. Get Term GUID
termGuid = None
for term in glossary[0]['terms']:
    if term['displayText'] == GLOSSARY_TERM_FORMAL_NAME:
        termGuid = term['termGuid']

In [ ]:
# 3. Convert CSV to JSON Payload
import csv
payload = []
with open(PATH_TO_CSV) as fp:
    reader = csv.reader(fp, delimiter=",", quotechar='"')
    next(reader, None)
    for row in reader:
        item = {"guid": row[0]}
        payload.append(item)
with open("payload.json", "w") as outfile:
    json.dump(payload, outfile, indent=4, sort_keys=False)

In [ ]:
# 4. Bulk assign Term to Assets
!pv glossary createTermsAssignedEntities --termGuid {termGuid} --payloadFile "payload.json"